In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import string
import time
import datetime
from datetime import datetime
from datetime import date
from scipy.stats import linregress

In [4]:
#Read file
plane = pd.read_csv('Airplane_Crashes_and_Fatalities.csv')
plane.head()

,Date,Time,Location,Operator,Flight #,Route,AC Type,Registration,cn/ln,Aboard,Aboard Passangers,Aboard Crew,Fatalities,Fatalities Passangers,Fatalities Crew,Ground,Summary
0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,1.0,1.0,1.0,0.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,09/07/1909,NaN,"Juvisy-sur-Orge, France",NaN,NaN,Air show,Wright Byplane,SC1,NaN,1.0,0.0,1.0,1.0,0.0,0.0,0.0,Eugene Lefebvre was the first pilot to ever be...
2,07/12/1912,06:30,"Atlantic City, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,0.0,5.0,5.0,0.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
3,08/06/1913,NaN,"Victoria, British Columbia, Canada",Private,NaN,NaN,Curtiss seaplane,NaN,NaN,1.0,0.0,1.0,1.0,0.0,1.0,0.0,The first fatal airplane accident in Canada oc...
4,09/09/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,NaN,NaN,14.0,NaN,NaN,0.0,The airship flew into a thunderstorm and encou...


In [5]:
#Add column year and extract data from Date.
Year = ""
plane['Year'] = pd.DatetimeIndex(plane.Date).year
plane.head()

,Date,Time,Location,Operator,Flight #,Route,AC Type,Registration,cn/ln,Aboard,Aboard Passangers,Aboard Crew,Fatalities,Fatalities Passangers,Fatalities Crew,Ground,Summary,Year
0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,1.0,1.0,1.0,0.0,0.0,"During a demonstration flight, a U.S. Army fly...",1908
1,09/07/1909,NaN,"Juvisy-sur-Orge, France",NaN,NaN,Air show,Wright Byplane,SC1,NaN,1.0,0.0,1.0,1.0,0.0,0.0,0.0,Eugene Lefebvre was the first pilot to ever be...,1909
2,07/12/1912,06:30,"Atlantic City, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,0.0,5.0,5.0,0.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...,1912
3,08/06/1913,NaN,"Victoria, British Columbia, Canada",Private,NaN,NaN,Curtiss seaplane,NaN,NaN,1.0,0.0,1.0,1.0,0.0,1.0,0.0,The first fatal airplane accident in Canada oc...,1913
4,09/09/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,NaN,NaN,14.0,NaN,NaN,0.0,The airship flew into a thunderstorm and encou...,1913


In [6]:
#break data down by year
year = plane[(plane.Year >= 1980) & (plane.Year <= 2019)]
year.head()

,Date,Time,Location,Operator,Flight #,Route,AC Type,Registration,cn/ln,Aboard,Aboard Passangers,Aboard Crew,Fatalities,Fatalities Passangers,Fatalities Crew,Ground,Summary,Year
3047,01/02/1980,NaN,Haiti,Haïti Air Inter,NaN,Port au Prince - Cape Haitien,Britten Norman BN Islander,HH-CNC,474,9.0,8.0,1.0,9.0,8.0,1.0,0.0,Struck high ground in poor visibility while en...,1980
3048,01/11/1980,01:23,Over the Atlantic Ocean,Private charter,NaN,Shreveport - Baton Rouge,Cessna 441 Conquest,N442NC,NaN,2.0,1.0,1.0,2.0,1.0,1.0,0.0,The plane was on a 45 minute fight from Shreve...,1980
3049,01/14/1980,14:00,"Shungnak, Alaska",Ambler Air Service,NaN,"Ambler, AK - Kobuk, AK",Cessna 185A,N1690Z,185-0490,9.0,8.0,1.0,7.0,6.0,1.0,0.0,Crashed. Continued VFR flight into adverse wea...,1980
3050,01/21/1980,19:10,"Elburz Mtns., near Tehran, Iran",Iran Air,NaN,Babol Sar - Tehran,Boeing B-727-86,EP-IRD,19817/537,128.0,120.0,8.0,128.0,120.0,8.0,0.0,The aircraft crashed into a mountain during a ...,1980
3051,01/23/1980,NaN,"Mt. Cemonyet, Indonesia",Pelita Air Service,NaN,Bandung - Jakarta,CASA 212 Aviocar 100,PK-PCX,45/3N,13.0,10.0,3.0,13.0,10.0,3.0,0.0,"Hit high ground at 2,700 ft. in poor weather w...",1980


In [7]:
# Remove words [Off, Near, the, Over]
words = ["Near", "Off", "the", "Over", "near"]
for word in words:
    year.Location = year.Location.str.replace(word, "")

In [8]:
#Display Results
year.head()

,Date,Time,Location,Operator,Flight #,Route,AC Type,Registration,cn/ln,Aboard,Aboard Passangers,Aboard Crew,Fatalities,Fatalities Passangers,Fatalities Crew,Ground,Summary,Year
3047,01/02/1980,NaN,Haiti,Haïti Air Inter,NaN,Port au Prince - Cape Haitien,Britten Norman BN Islander,HH-CNC,474,9.0,8.0,1.0,9.0,8.0,1.0,0.0,Struck high ground in poor visibility while en...,1980
3048,01/11/1980,01:23,Atlantic Ocean,Private charter,NaN,Shreveport - Baton Rouge,Cessna 441 Conquest,N442NC,NaN,2.0,1.0,1.0,2.0,1.0,1.0,0.0,The plane was on a 45 minute fight from Shreve...,1980
3049,01/14/1980,14:00,"Shungnak, Alaska",Ambler Air Service,NaN,"Ambler, AK - Kobuk, AK",Cessna 185A,N1690Z,185-0490,9.0,8.0,1.0,7.0,6.0,1.0,0.0,Crashed. Continued VFR flight into adverse wea...,1980
3050,01/21/1980,19:10,"Elburz Mtns., Tehran, Iran",Iran Air,NaN,Babol Sar - Tehran,Boeing B-727-86,EP-IRD,19817/537,128.0,120.0,8.0,128.0,120.0,8.0,0.0,The aircraft crashed into a mountain during a ...,1980
3051,01/23/1980,NaN,"Mt. Cemonyet, Indonesia",Pelita Air Service,NaN,Bandung - Jakarta,CASA 212 Aviocar 100,PK-PCX,45/3N,13.0,10.0,3.0,13.0,10.0,3.0,0.0,"Hit high ground at 2,700 ft. in poor weather w...",1980


In [9]:
#Break remove unwanted columns, add Crash ID as primary key. 
Crash_Id = range(1920)
New = year[['Date', 'Location', 'Aboard', 'Fatalities','Summary', 'Year']]
New["Crash Id"]= Crash_Id
New.set_index('Crash Id', inplace=True)
New

,Date,Location,Aboard,Fatalities,Summary,Year
Crash Id,,,,,,
0,01/02/1980,Haiti,9.0,9.0,Struck high ground in poor visibility while en...,1980
1,01/11/1980,Atlantic Ocean,2.0,2.0,The plane was on a 45 minute fight from Shreve...,1980
2,01/14/1980,"Shungnak, Alaska",9.0,7.0,Crashed. Continued VFR flight into adverse wea...,1980
3,01/21/1980,"Elburz Mtns., Tehran, Iran",128.0,128.0,The aircraft crashed into a mountain during a ...,1980
4,01/23/1980,"Mt. Cemonyet, Indonesia",13.0,13.0,"Hit high ground at 2,700 ft. in poor weather w...",1980
...,...,...,...,...,...,...
1915,04/16/2019,"Puerto Montt, Chile",6.0,6.0,"While the aircraft was in the initial climb, p...",2019
1916,05/05/2019,"Monclava, Mexico",13.0,13.0,The aircraft crashed while en route on a retur...,2019
1917,05/05/2019,"Moscow, Russia",78.0,41.0,Forty-five minutes after taking off from Mosco...,2019


In [10]:
#extract Cities from Location column
def City(s):
    placeholder = s.find(',',0)
    ax = s[0:placeholder]
    if placeholder == (-1):
        return s 
    else:
        return ax

In [11]:
#display example
City('Shungnak, Alaska')

'Shungnak'

In [12]:
#identity and apply extraccted Data
New['City'] = New.Location.apply(City)
x = New[['Date', 'Location', 'City', 'Aboard', 'Fatalities', 'Summary', 'Year']]

In [13]:
#remove trailing spaces
x.City = x.City.str.strip()

In [14]:
#export to csv
x.to_csv('plane_crash.csv')

In [15]:
# display updates
x.head()

,Date,Location,City,Aboard,Fatalities,Summary,Year
Crash Id,,,,,,,
0,01/02/1980,Haiti,Haiti,9.0,9.0,Struck high ground in poor visibility while en...,1980
1,01/11/1980,Atlantic Ocean,Atlantic Ocean,2.0,2.0,The plane was on a 45 minute fight from Shreve...,1980
2,01/14/1980,"Shungnak, Alaska",Shungnak,9.0,7.0,Crashed. Continued VFR flight into adverse wea...,1980
3,01/21/1980,"Elburz Mtns., Tehran, Iran",Elburz Mtns.,128.0,128.0,The aircraft crashed into a mountain during a ...,1980
4,01/23/1980,"Mt. Cemonyet, Indonesia",Mt. Cemonyet,13.0,13.0,"Hit high ground at 2,700 ft. in poor weather w...",1980


In [16]:
#list dates
dates = x.Date.to_list()
print(dates)

/1989', '06/18/1989', '06/22/1989', '06/27/1989', '06/28/1989', '06/28/1989', '06/29/1989', '07/19/1989', '07/19/1989', '07/19/1989', '07/21/1989', '07/21/1989', '07/23/1989', '07/26/1989', '07/27/1989', '07/27/1989', '07/31/1989', '08/03/1989', '08/07/1989', '08/08/1989', '08/13/1989', '08/15/1989', '08/25/1989', '09/03/1989', '09/03/1989', '09/08/1989', '09/15/1989', '09/15/1989', '09/19/1989', '09/20/1989', '09/20/1989', '09/21/1989', '09/21/1989', '09/23/1989', '09/26/1989', '09/27/1989', '09/28/1989', '10/04/1989', '10/06/1989', '10/10/1989', '10/10/1989', '10/18/1989', '10/20/1989', '10/21/1989', '10/26/1989', '10/26/1989', '10/28/1989', '11/09/1989', '11/10/1989', '11/21/1989', '11/23/1989', '11/25/1989', '11/27/1989', '11/27/1989', '11/28/1989', '11/28/1989', '12/01/1989', '12/21/1989', '12/26/1989', '12/28/1989', '12/30/1989', '01/02/1990', '01/13/1990', '01/15/1990', '01/17/1990', '01/23/1990', '01/25/1990', '01/25/1990', '01/26/1990', '01/27/1990', '01/29/1990', '01/29/1990'

In [17]:
#convert dates to UTC
UTC = []
for date in dates:
    t =datetime.strptime(date, "%m/%d/%Y")
    t =time.mktime(t.timetuple())
    UTC.append(int(t))

In [18]:
# Display reasults
x["Utc"]= UTC
x= x[['Date', 'Utc','Location', 'City', 'Aboard', 'Fatalities', 'Summary', 'Year']]
x

,Date,Utc,Location,City,Aboard,Fatalities,Summary,Year
Crash Id,,,,,,,,
0,01/02/1980,315640800,Haiti,Haiti,9.0,9.0,Struck high ground in poor visibility while en...,1980
1,01/11/1980,316418400,Atlantic Ocean,Atlantic Ocean,2.0,2.0,The plane was on a 45 minute fight from Shreve...,1980
2,01/14/1980,316677600,"Shungnak, Alaska",Shungnak,9.0,7.0,Crashed. Continued VFR flight into adverse wea...,1980
3,01/21/1980,317282400,"Elburz Mtns., Tehran, Iran",Elburz Mtns.,128.0,128.0,The aircraft crashed into a mountain during a ...,1980
4,01/23/1980,317455200,"Mt. Cemonyet, Indonesia",Mt. Cemonyet,13.0,13.0,"Hit high ground at 2,700 ft. in poor weather w...",1980
...,...,...,...,...,...,...,...,...
1915,04/16/2019,1555390800,"Puerto Montt, Chile",Puerto Montt,6.0,6.0,"While the aircraft was in the initial climb, p...",2019
1916,05/05/2019,1557032400,"Monclava, Mexico",Monclava,13.0,13.0,The aircraft crashed while en route on a retur...,2019
1917,05/05/2019,1557032400,"Moscow, Russia",Moscow,78.0,41.0,Forty-five minutes after taking off from Mosco...,2019


In [19]:
#update city column from exported csv.
y = pd.read_csv('year.csv')
x['City']= y.City
x

,Date,Utc,Location,City,Aboard,Fatalities,Summary,Year
Crash Id,,,,,,,,
0,01/02/1980,315640800,Haiti,Port au Prince,9.0,9.0,Struck high ground in poor visibility while en...,1980
1,01/11/1980,316418400,Atlantic Ocean,Baton Rouge,2.0,2.0,The plane was on a 45 minute fight from Shreve...,1980
2,01/14/1980,316677600,"Shungnak, Alaska",Shungnak,9.0,7.0,Crashed. Continued VFR flight into adverse wea...,1980
3,01/21/1980,317282400,"Elburz Mtns., Tehran, Iran",Elburz Mtns.,128.0,128.0,The aircraft crashed into a mountain during a ...,1980
4,01/23/1980,317455200,"Mt. Cemonyet, Indonesia",Mt. Cemonyet,13.0,13.0,"Hit high ground at 2,700 ft. in poor weather w...",1980
...,...,...,...,...,...,...,...,...
1915,04/16/2019,1555390800,"Puerto Montt, Chile",Puerto Montt,6.0,6.0,"While the aircraft was in the initial climb, p...",2019
1916,05/05/2019,1557032400,"Monclava, Mexico",Monclava,13.0,13.0,The aircraft crashed while en route on a retur...,2019
1917,05/05/2019,1557032400,"Moscow, Russia",Moscow,78.0,41.0,Forty-five minutes after taking off from Mosco...,2019


In [20]:
#converting UTC to date time example
pd.to_datetime(1490195805, unit='s')

Timestamp('2017-03-22 15:16:45')

In [40]:
#Selected random US cities for data frame
cit = pd.read_csv('selected_cities.csv').dropna()
cit

,Crash Id,Date,Location,City,Aboard,Fatalities,Summary,Year
0,41.0,11/28/1980,"Pecos City, Texas",Pecos City,2.0,2.0,The cargo plane entered a steep bank shortly a...,1980.0
1,48.0,2/11/1981,"White Plains, New York",White Plains,8.0,8.0,"Crashed in rain, fog, windshear. Pilot distra...",1981.0
2,108.0,3/19/1982,"Woodstock, Illinois",Woodstock,27.0,27.0,Crashed 40 miles northwest of Chicago O'Hare A...,1982.0
3,150.0,1/11/1983,"Detroit, Michigan",Detroit,3.0,3.0,After taking off the cargo plane rolled to the...,1983.0
4,216.0,6/19/1984,"Island Beach, New Jersey",Island Beach,3.0,3.0,The on-demand air taxi flight departed for gar...,1984.0
5,249.0,1/20/1985,"Carson, New Mexico",Carson,3.0,3.0,The helicopter crashed in open terrain during ...,1985.0
6,308.0,3/13/1986,"Alpena, Michigan",Alpena,9.0,3.0,"After a missed approach due to bad weather, th...",1986.0
7,354.0,1/15/1987,"Salt Lake City, Utah",Salt Lake City,10.0,10.0,"Midair collision with a Mooney M-20 at 7,000 f...",1987.0
8,414.0,2/19/1988,"Cary, North Carolina",Cary,12.0,12.0,The plane crashed into a reservoir shortly aft...,1988.0
9,470.0,1/31/1989,"Abilene, Texas",Abilene,19.0,19.0,Crashed in flames after taking off from Dyess ...,1989.0


In [36]:
#Pull data from underground weather
weather = pd.read_csv('weather.csv')
weather

,date,city,temp,pressure,humidity,wind_speed,weather_summary,visibility,cloudiness
0,11/28/1980,"Pecos City, TX",44.25,27.17,41,17,Fair,20,False
1,3/21/1980,"Houston, TX",57.89,30.22,33,21,Fair,15,False
2,6/12/1980,"Valley, NE",65.11,28.99,76,22,Mostly Cloudy,10,True
3,2/11/1981,"White Plains, NY",38.92,29.92,90,20,cloudy,10,True
4,1/11/1983,"Detroit, MI",32.18,29.10,76,16,cloudy,15,True
5,12/3/1990,"Romulus, MI",22.02,29.70,73,28,mostly cloudy,15,True
6,2/17/1991,"Cleveland, OH",23.34,29.12,49,17,cloudy,10,True
7,3/16/1991,"San Diego, CA",51.08,30.11,64,13,fair,20,False
8,3/31/1993,"Anchorage, AK",39.54,29.48,31,21,cloudy,60,True
9,7/12/1993,"Talladega, AL",55.91,29.35,79,9,cloudy,15,True
